<div style="padding:30px; color: white; background-color: #0071CD">
<center>
<img src="img/logoub.jpeg"></img>
<center>
<h1>Algorísmica Avançada 2022</h1>
<h2>Problemes 10 - Enumeratius: Ramificació i Poda</h2>
</center>
</div>

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import numpy as np
from queue import PriorityQueue

<div class="alert alert-success">
    <h1>Problema 1: Sliding-Puzzle</h1>
    <br>
    Donat un taulell de $n\times n$ de nombres des d'$1$ fins a $n^2-1$ inicialment desordenats, volem trobar el nombre mínim de moviments possibles de manera que els nombres estiguin ordenats en ordre creixent i la casella sense número estigui a l'última posició.
</div>

<img src="https://www.researchgate.net/profile/Ruo-Ando/publication/347300656/figure/fig1/AS:969204928901121@1608087870493/Initial-state-and-goal-state-of-8-puzzle.ppm" width='25%'/>

In [3]:
from npuzzle import NPuzzle

# Inicialitzem un tauler i el barregem
board = NPuzzle()
board.create_board(n=3, moves=100)

print ("Un tauler aleatori:")
print(board)

Un tauler aleatori:
+-----------+
| 4 | 3 | 6 | 
+-----------+
| 2 | 1 | 8 | 
+-----------+
| 7 | 5 |   | 
+-----------+



La funció ``get_state_id`` ens retorna un 'string' amb la configuració del tauler. D'aquesta forma podrem guardar els estats que ja hem visitat per a no repetir-los


In [4]:
board.get_state_id()

'4,3,6,2,1,8,7,5,0'

Per a cada tauler, podem definir una cota que depèn del nombre de moviments que hem fet fins el moment i un valor optimista calculat com una 'distància' entre el tauler que estem considerant i el tauler objectiu.

$$C(X) = g(X) + h(X)$$
+ $g(X)$ és el nombre de passos que portem fins el moment.
+ $h(X)$ pot ser:
    + $h_1(X)$: El nombre de caselles que no estan al seu lloc sense tenir en compte la casella buida (hamming_distance)
    + $h_2(X)$: La suma de les distàncies de manhattan de cada casella al seu lloc correcte (manhattan_distance)

In [5]:
board.hamming_distance() # h1(X)

7

In [6]:
board.manhattan_distance() # h2(X)

10

Podem demanar quins moviments són valids des d'una configuració del tauler amb la funció ``allowed_moves()``. Un moviment consisteix en 'moure' la casella buida en una de les quatre direccions permeses:<br>
+ $L$: Left
+ $R$: Right
+ $U$: Up
+ $D$: Down

In [7]:
am = board.allowed_moves()
print(am)

['L', 'U']


Executem un moviment amb la funció ``move()``

In [8]:
new_board = board.move(am[0])
print(new_board)

+-----------+
| 4 | 3 | 6 | 
+-----------+
| 2 | 1 | 8 | 
+-----------+
| 7 |   | 5 | 
+-----------+



La funció state ens comprova si el nostre estat és un estat solució.

In [9]:
# Solucionat: True
# No solucionat: False
new_board.state()

False

In [16]:
import numpy as np

def solve_puzzle(board):
    # Millor solució trobada. Inicialment té cota infinit
    best_bound = np.inf
    best_board = board
    # Guardem en una cua de prioritat els taulells.
    # Guardarem les variables:
    # 1. Distància mínima (cota inferior) entre el tauler actual i el tauler solució
    # 2. Número de passes que duem en aquest tauler, g(X)
    # 3. El tauler
    pq = PriorityQueue()
    pq.put((board.manhattan_distance(), 0, board))
    # Com que els estats poden repetir-se al llarg de l'exploració, guardarem en un 'set' tots els
    # estats visitats. Així evidem tornar a visitar estats.
    existent_states = set([board.get_state_id()])
    expanded = 0
    while not pq.empty():
        # Obtenim un nou element de la cua
        curr_bound, curr_steps, curr_board = pq.get()
        expanded += 1
        # Mirem tots els moviments valids que podem fer des d'aquest tauler
        for a_move in curr_board.allowed_moves():
            new_board = curr_board.move(a_move)
            new_steps = curr_steps + 1
            new_bound = new_steps + new_board.manhattan_distance() # g(X) + h(X)
            # Si és un estat solució i ens millora la cota, actualitzem.
            if new_board.state():
                if new_bound < best_bound:
                    best_bound = new_bound
                    best_board = new_board
            # En cas de que no sigui solució però ens millori la cota. 
            elif (new_bound < best_bound) and (new_board.get_state_id() not in existent_states):
                existent_states.add(new_board.get_state_id())
                pq.put((new_bound,new_steps,new_board))   
    return best_bound, best_board, expanded
    

In [17]:
board = NPuzzle()
board.create_board(moves=100, n=3)
print("Tauler inicial:")
print(board)
distance, final_board, expanded = solve_puzzle(board)
print(f"Solucionat en {distance} passos")
print(f"Taulers expandits: {expanded}")
print("Tauler final:")
print(final_board)

Tauler inicial:
+-----------+
| 6 | 7 | 5 | 
+-----------+
| 1 |   | 3 | 
+-----------+
| 2 | 8 | 4 | 
+-----------+

Solucionat en 24 passos
Taulers expandits: 3256
Tauler final:
+-----------+
| 1 | 2 | 3 | 
+-----------+
| 4 | 5 | 6 | 
+-----------+
| 7 | 8 |   | 
+-----------+

